In [1]:
# Install dependencies
!pip install -q unsloth datasets transformers accelerate bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# Import libraries
from datasets import load_dataset
#Import SFTTrainer from unsloth.trainer
from unsloth.trainer import SFTTrainer
from transformers import TrainingArguments
import torch

In [5]:
# Load Gemma 9B with Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2b-it",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.3.19: Fast Gemma patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [25]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("squad")

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

# Define the tokenization function
def tokenize_function(examples):
    return tokenizer(examples["context"], examples["question"], truncation=True, padding="max_length", max_length=2048)

# Apply the tokenization function to the dataset
dataset = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [26]:
# Ensure formatting
def formatting_func(example):
    context = example.get("context", "")
    question = example.get("question", "")

    try:
        if isinstance(example["answers"], dict):
            answer_list = example["answers"].get("text", [])
        elif isinstance(example["answers"], list):
            answer_list = example["answers"][0].get("text", []) if example["answers"] else []
        else:
            answer_list = []

        answer = answer_list[0] if answer_list else "No answer"
    except Exception as e:
        answer = "No answer"

    # Return a list containing the formatted string
    return [f"""Context: {context}
Question: {question}
Answer: {answer}"""]

In [27]:
# Add LoRA to model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [30]:
# Now apply to the model training as before
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    formatting_func=formatting_func,
    dataset_text_field=None,
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_steps=200,
        output_dir="qa_outputs",
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        report_to="none",
    ),
)

In [31]:
# Train the model
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 87,599 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,611,648/2,000,000,000 (0.98% trained)


Step,Training Loss
10,9.181200
20,9.244700
30,9.214700
40,9.127200
50,9.039400
60,8.875300
70,8.852100
80,8.670700
90,8.412000
100,8.331800


TrainOutput(global_step=100, training_loss=8.89492332458496, metrics={'train_runtime': 1550.6976, 'train_samples_per_second': 0.516, 'train_steps_per_second': 0.064, 'total_flos': 1.96755069075456e+16, 'train_loss': 8.89492332458496})

In [32]:
# Save final model
model.save_pretrained("fine-tuned-gemma-qa")
tokenizer.save_pretrained("fine-tuned-gemma-qa")

('fine-tuned-gemma-qa/tokenizer_config.json',
 'fine-tuned-gemma-qa/special_tokens_map.json',
 'fine-tuned-gemma-qa/vocab.txt',
 'fine-tuned-gemma-qa/added_tokens.json',
 'fine-tuned-gemma-qa/tokenizer.json')